In [1]:
#Importar librerias para procesamiento de datos

import pandas as pd
import numpy as np

In [2]:
from scipy.spatial import KDTree

In [3]:
from numpy import cos, sin, arcsin, sqrt
from math import radians

In [18]:
df_clusteres2

,x,y,index_vertice_cercano
0,4.124433,-73.627487,33483
1,4.118694,-73.563922,37514
2,4.075202,-73.672954,52077
3,4.153048,-73.599674,11110
4,4.148361,-73.613292,14637
...,...,...,...
195,4.158273,-73.651119,7341
196,4.103017,-73.659508,45497
197,4.155809,-73.632643,8546
198,4.136147,-73.609554,23612


In [9]:
df_clusteres = pd.read_csv("clusteresF.csv", delimiter=",")

In [5]:
df_vertices = pd.read_csv("vertices.csv", delimiter=";")

In [7]:
df_vertices2 = df_vertices[['LATITUD', 'LONGITUD']].copy()

In [14]:
df_clusteres2 = df_clusteres[['x', 'y']].copy()

In [15]:
tree = KDTree(df_vertices2)

In [17]:
%%time
distancias, indices = tree.query(df_clusteres2, k=1)
df_clusteres2['index_vertice_cercano'] = indices

Wall time: 2.99 ms


In [24]:
merged_dataframes

,x,y,index_vertice_cercano,LATITUD,LONGITUD,distancia_h
0,4.124433,-73.627487,33483,4.124,-73.627545,0.048541
1,4.118694,-73.563922,37514,4.119,-73.563647,0.045658
2,4.075202,-73.672954,52077,4.075,-73.672891,0.023477
3,4.153048,-73.599674,11110,4.153,-73.599651,0.005913
4,4.148361,-73.613292,14637,4.148,-73.613417,0.042406
...,...,...,...,...,...,...
195,4.158273,-73.651119,7341,4.158,-73.651163,0.030774
196,4.103017,-73.659508,45497,4.103,-73.659841,0.036985
197,4.155809,-73.632643,8546,4.156,-73.632766,0.025231
198,4.136147,-73.609554,23612,4.136,-73.609601,0.017196


In [19]:
merged_dataframes = pd.merge(df_clusteres2, df_vertices2, left_on='index_vertice_cercano', right_index=True)

In [21]:
def haversine(row):
    lon1 = row['y']
    lat1 = row['x']
    lon2 = row['LONGITUD']
    lat2 = row['LATITUD']
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a)) 
    km = 6367 * c
    return km

In [22]:
merged_dataframes['distancia_h'] = merged_dataframes.apply(lambda row: haversine(row), axis=1)

In [23]:
merged_dataframes.distancia_h.describe()

count    200.000000
mean       0.056871
std        0.179300
min        0.003008
25%        0.017699
50%        0.035098
75%        0.046083
max        1.937663
Name: distancia_h, dtype: float64

In [25]:
merged_dataframes.to_csv("Clusters+vert_cercanos.csv",index=True)